In [1]:
import torch
import torchvision
from torchvision.transforms import TrivialAugmentWide,AugMix,AutoAugment,RandAugment,RandomHorizontalFlip,Resize,ToTensor,Compose,RandomCrop,RandomRotation,Normalize,CenterCrop
from torch.utils.data import DataLoader,Dataset, random_split
#transform=torchvision.transforms.RandomHorizontalFlip()
#transform=torchvision.transforms.RandomAffine(degrees=0,translate=(0.5,0.5))
#dataset=torchvision.datasets.ImageFolder("flowers",transform=transform)
dataset=torchvision.datasets.ImageFolder("flower_photos")

In [2]:
seed=9 #67 after 30 epochs with no aug.

torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic=True

In [3]:
class MyDataset(Dataset):
    def __init__(self,subset,transform=None):
        self.subset=subset
        self.transform=transform
    def __getitem__(self,idx):
        x,y=self.subset[idx]
        if self.transform:
            x=self.transform(x)
        return x,y
    def __len__(self):
        return len(self.subset)

In [4]:
len(dataset)

3670

In [5]:
#train_d,valid_d,test_d=random_split(dataset,lengths=[0.7,0.2,0.1])
train_d,valid_d=random_split(dataset,lengths=[0.8,0.2])


In [6]:
augment_choices={'horizontal':RandomHorizontalFlip(0.8),'trivial':TrivialAugmentWide(),'random':RandAugment(),'mix':AugMix(),'auto':AutoAugment()}
#train_t=Compose([RandomHorizontalFlip(),Resize((180,180)),ToTensor()])
augmentation='trivial'
valid_t=Compose([Resize((180,180)),ToTensor()])
if augmentation=='None':
    train_t=valid_t
else:
    train_t=Compose([augment_choices[augmentation],Resize((180,180)),ToTensor()])

In [7]:
train_dt=MyDataset(train_d,train_t)
valid_dt=MyDataset(valid_d,valid_t)
#test_dt=MyDataset(test_d,test_t)

In [8]:
train_loader=DataLoader(train_dt,batch_size=32,shuffle=True,num_workers=2)
valid_loader=DataLoader(valid_dt,batch_size=32,shuffle=False,num_workers=2)

In [9]:
itr=iter(train_loader)
imgs,labels=next(itr)

In [10]:
from torch import nn
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.relu=nn.ReLU()
        #[3, 224, 224]
        self.conv1=nn.Conv2d(in_channels=3,out_channels=16,kernel_size=3)
        # [16,222,222]
        self.conv2=nn.Conv2d(in_channels=16,out_channels=16,kernel_size=3)
        #[16,220,220]
        self.pool1=nn.MaxPool2d(kernel_size=2)
        #[16,110,110]
        self.norm1=nn.BatchNorm2d(num_features=16)
        
        self.conv3=nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3)
         #[32,108,108]
        self.conv4=nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3)
        #[32,106,106]
        self.pool2=nn.MaxPool2d(kernel_size=2)
        #[32,53,53]
        self.norm2=nn.BatchNorm2d(num_features=32)
        
        
        self.conv5=nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3)
        self.conv6=nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3,padding="same")
         #[32,51,51]
        self.pool3=nn.MaxPool2d(kernel_size=2)
        #[32,25,25]
        self.norm3=nn.BatchNorm2d(num_features=32)
        
        #self.fc1=nn.Linear(in_features=89888,out_features=256)
        self.fc1=nn.Linear(in_features=20000,out_features=256)
        self.fc2=nn.Linear(in_features=256,out_features=5)
    def forward(self,x):
        x=self.conv1(x)
        x=self.relu(x)
        x=self.conv2(x)
        x=self.relu(x)
        x=self.pool1(x)
       # x=self.norm1(x)
        
        x=self.conv3(x)
        x=self.relu(x)
        x=self.conv4(x)
        x=self.relu(x)
        x=self.pool2(x)
       # x=self.norm2(x)
        
        x=self.conv5(x)
        x=self.relu(x)
        x=self.conv6(x)
        x=self.relu(x)
        x=self.pool3(x)
        #x=self.norm3(x)
        x=x.flatten(start_dim=1)
        x=self.fc1(x)
        x=self.relu(x)
        x=self.fc2(x)
        return x
        
        

In [11]:
class Model2(nn.Module):
    def __init__(self):
        super().__init__()
        #(3,180,180)
        self.conv1=nn.Conv2d(in_channels=3,out_channels=16,kernel_size=3,padding='same')
        self.pool1=nn.MaxPool2d(2)
        #(16,90,90)
        self.conv2=nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,padding='same')
        self.pool2=nn.MaxPool2d(2)
        #(32,45,45)
        self.conv3=nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,padding='same')
        self.pool3=nn.MaxPool2d(2)
        #(64,22,22)
        self.relu=nn.ReLU()
        self.drop=nn.Dropout(0.2)
        self.fc1=nn.Linear(in_features=64*22*22,out_features=128)
        self.fc2=nn.Linear(in_features=128,out_features=5)
    def forward(self,x):
        x=self.conv1(x)
        x=self.relu(x)
        x=self.pool1(x)
        x=self.conv2(x)
        x=self.relu(x)
        x=self.pool2(x)
        x=self.conv3(x)
        x=self.relu(x)
        x=self.pool3(x)
        #x=self.drop(x)
        x=x.view(x.size()[0],-1)
        x=self.fc1(x)
        x=self.relu(x)
        x=self.fc2(x)
        return x



In [12]:
model=Model2()

In [13]:
def accuracy(model,batch,loss_fn):
    imgs,labels=batch
    imgs=imgs.cuda()
    labels=labels.cuda()
    outputs=model(imgs)
    _,pred=torch.max(outputs,dim=1)
    acc=torch.sum(pred==labels).item()
    loss=loss_fn(outputs,labels)
    return loss,torch.tensor(acc),len(labels)

@torch.no_grad() 
def evaluate(model,loader,loss_fn):
    model.eval()
    # crit is a list of pairs of tensors
    crit=[accuracy(model,batch,loss_fn) for batch in loader]
    crit=torch.tensor(crit)
    m=crit.mean(dim=0)
    loss=m[0]
    acc=m[1]
    bsize=m[2]
    return loss/bsize,acc/bsize

In [14]:
from torch.utils.tensorboard import SummaryWriter
import datetime
%load_ext tensorboard
current=datetime.datetime.now()
log_dir = 'logs/tensorboard/' +augmentation+'-'+current.strftime("%c")
writer=SummaryWriter(log_dir)

In [15]:
from torch.optim import Adam
loss_fn=nn.CrossEntropyLoss()
optimizer=Adam(model.parameters())
model=model.cuda()
epochs=1000
for epoch in range(epochs):
    model.train()
    epoch_loss=0.
    for imgs,labels in train_loader:
        imgs=imgs.cuda()
        labels=labels.cuda()
        outputs=model(imgs)
        loss=loss_fn(outputs,labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        epoch_loss=0.9*epoch_loss+0.1*loss.item()
    vl,va=evaluate(model,valid_loader,loss_fn)
    tl,ta=evaluate(model,train_loader,loss_fn)
    writer.add_scalar("Epoch loss",epoch_loss,epoch)
    writer.add_scalars("loss",{'train':tl,'valid':vl},epoch)
    writer.add_scalars("acc",{'train':ta,'valid':va},epoch)
    if epoch%10==0:
        print("Epoch[{}]".format(epoch))
        print("Train: loss={:.4f}, accuracy={:.4f}".format(tl,ta))
        print("Valid: loss={:.4f}, accuracy={:.4f}".format(vl,va))
        print("----------------------")
writer.close()   

Epoch[0]
Train: loss=0.0385, accuracy=0.4693
Valid: loss=0.0358, accuracy=0.5123
----------------------
Epoch[10]
Train: loss=0.0227, accuracy=0.7231
Valid: loss=0.0261, accuracy=0.6839
----------------------
Epoch[20]
Train: loss=0.0133, accuracy=0.8501
Valid: loss=0.0268, accuracy=0.7234
----------------------
Epoch[30]
Train: loss=0.0117, accuracy=0.8713
Valid: loss=0.0293, accuracy=0.7384
----------------------
Epoch[40]
Train: loss=0.0105, accuracy=0.8753
Valid: loss=0.0339, accuracy=0.7221
----------------------
Epoch[50]
Train: loss=0.0095, accuracy=0.8965
Valid: loss=0.0276, accuracy=0.7330
----------------------
Epoch[60]
Train: loss=0.0080, accuracy=0.9097
Valid: loss=0.0317, accuracy=0.7044
----------------------
Epoch[70]
Train: loss=0.0070, accuracy=0.9223
Valid: loss=0.0322, accuracy=0.7452
----------------------
Epoch[80]
Train: loss=0.0078, accuracy=0.9142
Valid: loss=0.0330, accuracy=0.7275
----------------------
Epoch[90]
Train: loss=0.0074, accuracy=0.9176
Valid: los

In [16]:
%tensorboard --logdir logs/tensorboard

Reusing TensorBoard on port 6007 (pid 2216015), started 1 day, 19:24:05 ago. (Use '!kill 2216015' to kill it.)